<!-- ![](https://europe-west1-atp-views-tracker.cloudfunctions.net/working-analytics?notebook=adaptive-retrieval) -->



# Adaptive Retrieval-Augmented Generation (RAG) System

## Overview

This system implements an advanced Retrieval-Augmented Generation (RAG) approach that adapts its retrieval strategy based on the type of query. By leveraging Language Models (LLMs) at various stages, it aims to provide more accurate, relevant, and context-aware responses to user queries.

## Motivation

Traditional RAG systems often use a one-size-fits-all approach to retrieval, which can be suboptimal for different types of queries. Our adaptive system is motivated by the understanding that different types of questions require different retrieval strategies. For example, a factual query might benefit from precise, focused retrieval, while an analytical query might require a broader, more diverse set of information.

## Key Components

1. **Query Classifier**: Determines the type of query (Factual, Analytical, Opinion, or Contextual).

2. **Adaptive Retrieval Strategies**: Four distinct strategies tailored to different query types:
   - Factual Strategy
   - Analytical Strategy
   - Opinion Strategy
   - Contextual Strategy

3. **LLM Integration**: LLMs are used throughout the process to enhance retrieval and ranking.

4. **OpenAI GPT Model**: Generates the final response using the retrieved documents as context.

## Method Details

### 1. Query Classification

The system begins by classifying the user's query into one of four categories:
- Factual: Queries seeking specific, verifiable information.
- Analytical: Queries requiring comprehensive analysis or explanation.
- Opinion: Queries about subjective matters or seeking diverse viewpoints.
- Contextual: Queries that depend on user-specific context.

### 2. Adaptive Retrieval Strategies

Each query type triggers a specific retrieval strategy:

#### Factual Strategy
- Enhances the original query using an LLM for better precision.
- Retrieves documents based on the enhanced query.
- Uses an LLM to rank documents by relevance.

#### Analytical Strategy
- Generates multiple sub-queries using an LLM to cover different aspects of the main query.
- Retrieves documents for each sub-query.
- Ensures diversity in the final document selection using an LLM.

#### Opinion Strategy
- Identifies different viewpoints on the topic using an LLM.
- Retrieves documents representing each viewpoint.
- Uses an LLM to select a diverse range of opinions from the retrieved documents.

#### Contextual Strategy
- Incorporates user-specific context into the query using an LLM.
- Performs retrieval based on the contextualized query.
- Ranks documents considering both relevance and user context.

### 3. LLM-Enhanced Ranking

After retrieval, each strategy uses an LLM to perform a final ranking of the documents. This step ensures that the most relevant and appropriate documents are selected for the next stage.

### 4. Response Generation

The final set of retrieved documents is passed to an OpenAI GPT model, which generates a response based on the query and the provided context.

## Benefits of This Approach

1. **Improved Accuracy**: By tailoring the retrieval strategy to the query type, the system can provide more accurate and relevant information.

2. **Flexibility**: The system adapts to different types of queries, handling a wide range of user needs.

3. **Context-Awareness**: Especially for contextual queries, the system can incorporate user-specific information for more personalized responses.

4. **Diverse Perspectives**: For opinion-based queries, the system actively seeks out and presents multiple viewpoints.

5. **Comprehensive Analysis**: The analytical strategy ensures a thorough exploration of complex topics.

## Conclusion

This adaptive RAG system represents a significant advancement over traditional RAG approaches. By dynamically adjusting its retrieval strategy and leveraging LLMs throughout the process, it aims to provide more accurate, relevant, and nuanced responses to a wide variety of user queries.

![alt Adaptive Retrieval](<adaptive retrieval.png>)

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [ ]:
# Install required packages
!pip install Chroma langchain langchain-openai langchain-community pydantic

## Understanding the Required Libraries

Before we dive into the code, let's understand what each library does:

- **Chroma**: A vector database that stores our documents as embeddings (numerical representations) and helps us find similar documents quickly
- **LangChain**: A framework that makes it easier to build applications with language models, providing tools for document processing, prompts, and chains
- **OpenAI**: Provides access to OpenAI's language models like GPT-4 for text generation and embeddings
- **Pydantic**: Helps us define data structures with validation, ensuring our code handles data correctly

Think of these libraries as specialized tools in a toolbox - each one handles a specific part of our RAG system.

In [ ]:
import os
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_core.retrievers import BaseRetriever
from typing import List
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = "<your-api-key>"

## What is Query Classification?

Query classification is like having a smart assistant that can understand what type of question you're asking. Just as you might approach different types of questions differently in real life, our system needs to understand whether you're asking for:

- **Factual information**: "What is the capital of France?" or "How many planets are in our solar system?"
- **Analytical insights**: "How does climate change affect ocean currents?" or "What are the economic impacts of remote work?"
- **Multiple opinions**: "What do experts think about artificial intelligence?" or "What are different approaches to education?"
- **Context-specific answers**: "What's the best investment strategy for someone in their 20s?" or "How should I prepare for a job interview in tech?"

By classifying queries first, our system can choose the most appropriate retrieval strategy for each type of question.

### Define the query classifer class

In [ ]:
class categories_options(BaseModel):
        category: str = Field(description="The category of the query, the options are: Factual, Analytical, Opinion, or Contextual", example="Factual")


class QueryClassifier:
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000) #type: ignore
        self.prompt = PromptTemplate(
            input_variables=["query"],
            template="Classify the following query into one of these categories: Factual, Analytical, Opinion, or Contextual.\nQuery: {query}\nCategory:"
        )
        self.chain = self.prompt | self.llm.with_structured_output(categories_options)


    def classify(self, query):
        return self.chain.invoke(query).category #type: ignore

## How the Query Classifier Works

The QueryClassifier is like having a librarian who can instantly categorize any question you ask. Here's what happens:

1. **Input**: You give it a question like "What causes earthquakes?"
2. **Processing**: The classifier uses GPT-4 to analyze the question and determine its type
3. **Output**: It returns one of four categories: Factual, Analytical, Opinion, or Contextual

The `categories_options` class ensures the classifier always returns a valid category name, preventing errors in our system.

**Example classifications:**
- "What is the population of Tokyo?" → Factual
- "How do social media algorithms affect user behavior?" → Analytical
- "What do researchers think about the future of renewable energy?" → Opinion
- "What programming language should I learn as a beginner?" → Contextual

## Understanding Retrieval Strategies

Think of retrieval strategies as different ways of searching through a library. Just as you might use different approaches to find information depending on what you need, our system uses different strategies for different types of questions.

The BaseRetrievalStrategy is like the foundation of a building - it provides the basic tools that all other strategies will use:
- **Text splitting**: Breaks down long documents into manageable chunks
- **Embeddings**: Converts text into numbers that computers can understand and compare
- **Vector database**: Stores and searches through these numerical representations efficiently
- **Language model**: Provides intelligent processing capabilities

All our specialized strategies will inherit these basic capabilities and add their own unique approaches on top.

### Define the Base Retriever class, such that the complex ones will inherit from it

In [ ]:
class BaseRetrievalStrategy:
    def __init__(self, texts):
        self.embeddings = OpenAIEmbeddings()
        text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=0)
        self.documents = text_splitter.create_documents(texts)
        self.db = Chroma.from_documents(self.documents, embedding=self.embeddings, persist_directory=None)  # set to "./chroma_db" if you want persistence
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000) #type: ignore


    def retrieve(self, query, k=4):
        return self.db.similarity_search(query, k=k)

## The Factual Retrieval Strategy

When someone asks a factual question like "What is the boiling point of water?", they want a precise, accurate answer. The Factual strategy is designed specifically for these types of queries.

Here's how it works:

1. **Query Enhancement**: The system first improves the original question to make it more specific. For example, "boiling point water" might become "What is the boiling point of water at standard atmospheric pressure?"

2. **Document Retrieval**: It searches for documents using the enhanced query, getting more results than needed initially

3. **Intelligent Ranking**: Each retrieved document gets a relevance score from 1-10, helping identify the most accurate sources

4. **Best Results**: Finally, it returns only the top-ranked documents that are most likely to contain the correct factual information

This approach ensures that factual questions get precise, well-sourced answers rather than vague or tangentially related information.

### Define Factual retriever strategy

In [ ]:
class relevant_score(BaseModel):
        score: float = Field(description="The relevance score of the document to the query", example=8.0)

class FactualRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query, k=4):
        print("retrieving factual")
        # Use LLM to enhance the query
        enhanced_query_prompt = PromptTemplate(
            input_variables=["query"],
            template="Enhance this factual query for better information retrieval: {query}"
        )
        query_chain = enhanced_query_prompt | self.llm
        enhanced_query = query_chain.invoke(query).content
        print(f'enhande query: {enhanced_query}')

        # Retrieve documents using the enhanced query
        docs = self.db.similarity_search(enhanced_query, k=k*2) #type: ignore

        # Use LLM to rank the relevance of retrieved documents
        ranking_prompt = PromptTemplate(
            input_variables=["query", "doc"],
            template="On a scale of 1-10, how relevant is this document to the query: '{query}'?\nDocument: {doc}\nRelevance score:"
        )
        ranking_chain = ranking_prompt | self.llm.with_structured_output(relevant_score)

        ranked_docs = []
        print("ranking docs")
        for doc in docs:
            input_data = {"query": enhanced_query, "doc": doc.page_content}
            score = float(ranking_chain.invoke(input_data).score) #type: ignore
            ranked_docs.append((doc, score))

        # Sort by relevance score and return top k
        ranked_docs.sort(key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in ranked_docs[:k]]

## The Analytical Retrieval Strategy

Analytical questions require a comprehensive understanding of a topic from multiple angles. Think of questions like "How does artificial intelligence impact the job market?" - this isn't just asking for one fact, but needs analysis from various perspectives.

The Analytical strategy works like a research team:

1. **Breaking Down the Question**: It takes your complex question and creates several focused sub-questions. For our AI example, it might generate:
   - "Which jobs are most at risk from AI automation?"
   - "What new job categories are being created by AI?"
   - "How are wages affected in AI-impacted industries?"
   - "What skills are becoming more valuable due to AI?"

2. **Comprehensive Search**: Each sub-question is used to find relevant documents, ensuring broad coverage

3. **Diversity Selection**: The system then intelligently selects the most diverse and relevant documents, avoiding repetitive information

This approach ensures that analytical questions receive well-rounded, thorough responses that cover multiple aspects of the topic.

### Define Analytical reriever strategy

In [ ]:
class SelectedIndices(BaseModel):
    indices: List[int] = Field(description="Indices of selected documents", example=[0, 1, 2, 3])

class SubQueries(BaseModel):
    sub_queries: List[str] = Field(description="List of sub-queries for comprehensive analysis", example=["What is the population of New York?", "What is the GDP of New York?"])

class AnalyticalRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query, k=4):
        print("retrieving analytical")
        # Use LLM to generate sub-queries for comprehensive analysis
        sub_queries_prompt = PromptTemplate(
            input_variables=["query", "k"],
            template="Generate {k} sub-questions for: {query}"
        )

        llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000) #type: ignore
        sub_queries_chain = sub_queries_prompt | llm.with_structured_output(SubQueries)

        input_data = {"query": query, "k": k}
        sub_queries = sub_queries_chain.invoke(input_data).sub_queries #type: ignore
        print(f'sub queries for comprehensive analysis: {sub_queries}')

        all_docs = []
        for sub_query in sub_queries:
            all_docs.extend(self.db.similarity_search(sub_query, k=2))

        # Use LLM to ensure diversity and relevance
        diversity_prompt = PromptTemplate(
            input_variables=["query", "docs", "k"],
            template="""Select the most diverse and relevant set of {k} documents for the query: '{query}'\nDocuments: {docs}\n
            Return only the indices of selected documents as a list of integers."""
        )
        diversity_chain = diversity_prompt | self.llm.with_structured_output(SelectedIndices)
        docs_text = "\n".join([f"{i}: {doc.page_content[:50]}..." for i, doc in enumerate(all_docs)])
        input_data = {"query": query, "docs": docs_text, "k": k}
        selected_indices_result = diversity_chain.invoke(input_data).indices #type: ignore
        print(f'selected diverse and relevant documents')
        
        return [all_docs[i] for i in selected_indices_result if i < len(all_docs)]

## The Opinion Retrieval Strategy

Some questions don't have a single "correct" answer but instead benefit from multiple perspectives. Questions like "What do experts think about cryptocurrency?" or "How should we address climate change?" require understanding different viewpoints.

The Opinion strategy acts like a balanced journalist:

1. **Identifying Viewpoints**: First, it identifies the main perspectives that exist on the topic. For cryptocurrency, it might identify:
   - "Cryptocurrency as revolutionary financial technology"
   - "Cryptocurrency as speculative bubble"
   - "Cryptocurrency as regulatory challenge"

2. **Searching Each Perspective**: It then searches for documents that represent each viewpoint

3. **Balanced Selection**: Finally, it selects documents that provide a diverse range of opinions, ensuring no single viewpoint dominates

This approach helps users understand complex, debated topics by presenting multiple legitimate perspectives rather than a single biased view.

### Define Opinion retriever strategy

In [ ]:
class OpinionRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query, k=3):
        print("retrieving opinion")
        # Use LLM to identify potential viewpoints
        viewpoints_prompt = PromptTemplate(
            input_variables=["query", "k"],
            template="Identify {k} distinct viewpoints or perspectives on the topic: {query}"
        )
        viewpoints_chain = viewpoints_prompt | self.llm
        input_data = {"query": query, "k": k}
        viewpoints = viewpoints_chain.invoke(input_data).content.split('\n') #type: ignore
        print(f'viewpoints: {viewpoints}')

        all_docs = []
        for viewpoint in viewpoints:
            all_docs.extend(self.db.similarity_search(f"{query} {viewpoint}", k=2))

        # Use LLM to classify and select diverse opinions
        opinion_prompt = PromptTemplate(
            input_variables=["query", "docs", "k"],
            template="Classify these documents into distinct opinions on '{query}' and select the {k} most representative and diverse viewpoints:\nDocuments: {docs}\nSelected indices:"
        )
        opinion_chain = opinion_prompt | self.llm.with_structured_output(SelectedIndices)
        
        docs_text = "\n".join([f"{i}: {doc.page_content[:100]}..." for i, doc in enumerate(all_docs)])
        input_data = {"query": query, "docs": docs_text, "k": k}
        selected_indices = opinion_chain.invoke(input_data).indices #type: ignore
        print(f'selected diverse and relevant documents')
        
        return [all_docs[int(i)] for i in selected_indices.split() if i.isdigit() and int(i) < len(all_docs)]

## The Contextual Retrieval Strategy

Some questions only make sense when we consider the specific situation or context of the person asking. For example, "What's the best programming language to learn?" depends heavily on the person's background, goals, and current situation.

The Contextual strategy works like a personalized consultant:

1. **Understanding Context**: It takes into account any provided user context (like "I'm a beginner interested in web development" or "I'm a data scientist looking to expand my skills")

2. **Reformulating the Query**: It rewrites the question to include this context, making it more specific and targeted

3. **Context-Aware Search**: It searches for documents using the contextualized query

4. **Contextual Ranking**: When ranking results, it considers both relevance to the topic and appropriateness for the user's specific situation

This ensures that context-dependent questions receive personalized, relevant answers rather than generic responses that might not be helpful for the specific user.

### Define Contextual retriever strategy

In [ ]:
class ContextualRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query, k=4, user_context=None):
        print("retrieving contextual")
        # Use LLM to incorporate user context into the query
        context_prompt = PromptTemplate(
            input_variables=["query", "context"],
            template="Given the user context: {context}\nReformulate the query to best address the user's needs: {query}"
        )
        context_chain = context_prompt | self.llm
        input_data = {"query": query, "context": user_context or "No specific context provided"}
        contextualized_query = context_chain.invoke(input_data).content
        print(f'contextualized query: {contextualized_query}')

        # Retrieve documents using the contextualized query
        docs = self.db.similarity_search(contextualized_query, k=k*2) #type: ignore

        # Use LLM to rank the relevance of retrieved documents considering the user context
        ranking_prompt = PromptTemplate(
            input_variables=["query", "context", "doc"],
            template="Given the query: '{query}' and user context: '{context}', rate the relevance of this document on a scale of 1-10:\nDocument: {doc}\nRelevance score:"
        )
        ranking_chain = ranking_prompt | self.llm.with_structured_output(relevant_score)
        print("ranking docs")

        ranked_docs = []
        for doc in docs:
            input_data = {"query": contextualized_query, "context": user_context or "No specific context provided", "doc": doc.page_content}
            score = float(ranking_chain.invoke(input_data).score) #type: ignore
            ranked_docs.append((doc, score))


        # Sort by relevance score and return top k
        ranked_docs.sort(key=lambda x: x[1], reverse=True)

        return [doc for doc, _ in ranked_docs[:k]]

## Bringing It All Together: The Adaptive Retriever

Now we have our four specialized strategies, but we need a way to automatically choose which one to use for each question. The AdaptiveRetriever is like a smart dispatcher that:

1. **Receives any question** from a user
2. **Classifies the question** using our QueryClassifier
3. **Routes the question** to the appropriate strategy (Factual, Analytical, Opinion, or Contextual)
4. **Returns the results** from the chosen strategy

This means users don't need to worry about what type of question they're asking - the system automatically adapts to provide the best possible search approach for each query type.

**Example workflow:**
- User asks: "What is the capital of Japan?"
- System classifies: "Factual"
- System uses: FactualRetrievalStrategy
- Result: Precise, well-sourced answer about Tokyo

### Define the Adapive retriever class

In [ ]:
class AdaptiveRetriever:
    def __init__(self, texts: List[str]):
        self.classifier = QueryClassifier()
        self.strategies = {
            "Factual": FactualRetrievalStrategy(texts),
            "Analytical": AnalyticalRetrievalStrategy(texts),
            "Opinion": OpinionRetrievalStrategy(texts),
            "Contextual": ContextualRetrievalStrategy(texts)
        }

    def get_relevant_documents(self, query: str) -> List[Document]:
        category = self.classifier.classify(query)
        print(f"Query classified as: {category}")
        
        strategy = self.strategies[category]
        print(f"Using strategy: {strategy}")
        return strategy.retrieve(query)

## Making It Compatible with LangChain

LangChain has its own standard format for retrievers, so we need to create a wrapper that makes our AdaptiveRetriever compatible with LangChain's ecosystem. The PydanticAdaptiveRetriever is like a translator that allows our custom retriever to work seamlessly with other LangChain components.

This wrapper ensures that our adaptive retriever can be easily integrated into larger LangChain applications and follows the framework's conventions for data handling and method signatures.

### Define aditional retriever that inherits from langchain BaseRetriever 

In [ ]:
class PydanticAdaptiveRetriever(BaseRetriever):
    adaptive_retriever: AdaptiveRetriever = Field(exclude=True)

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str) -> List[Document]: #type: ignore
        return self.adaptive_retriever.get_relevant_documents(query)

    async def _aget_relevant_documents(self, query: str) -> List[Document]: #type: ignore
        return self.get_relevant_documents(query)

## The Complete Adaptive RAG System

Finally, we bring everything together into a complete Retrieval-Augmented Generation system. The AdaptiveRAG class combines:

- **Smart Retrieval**: Our adaptive retriever that chooses the best strategy for each question
- **Response Generation**: A powerful language model (GPT-4) that creates natural, helpful responses
- **Prompt Engineering**: A carefully crafted prompt that guides the language model to use the retrieved information effectively

**How it works end-to-end:**
1. **User asks a question**: "How does exercise affect mental health?"
2. **System classifies**: "Analytical" (requires comprehensive understanding)
3. **Adaptive retrieval**: Uses analytical strategy to find diverse, relevant documents
4. **Response generation**: GPT-4 uses the retrieved documents to create a comprehensive, well-sourced answer
5. **User receives**: A detailed response that covers multiple aspects of how exercise affects mental health

This creates a system that's both intelligent in how it searches for information and sophisticated in how it presents that information to users.

### Define the Adaptive RAG class

In [ ]:
class AdaptiveRAG:
    def __init__(self, texts: List[str]):
        adaptive_retriever = AdaptiveRetriever(texts)
        self.retriever = PydanticAdaptiveRetriever(adaptive_retriever=adaptive_retriever)
        self.llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000) #type: ignore
        
        # Create a custom prompt
        prompt_template = """Use the following pieces of context to answer the question at the end. 
        If you don't know the answer, just say that you don't know, don't try to make up an answer.

        {context}

        Question: {question}
        Answer:"""
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
        
        # Create the LLM chain
        self.llm_chain = prompt | self.llm
        
      

    def answer(self, query: str) -> str:
        docs = self.retriever.get_relevant_documents(query)
        input_data = {"context": "\n".join([doc.page_content for doc in docs]), "question": query}
        return self.llm_chain.invoke(input_data) #type: ignore

## Setting Up Our Test Data

Before we can test our adaptive RAG system, we need some sample documents to search through. For this demonstration, we're using a set of texts about Earth and space that represent different types of information:

- **Factual statements**: Basic facts that can answer direct questions
- **Analytical content**: Information that helps explain relationships and causes
- **Opinion/theoretical content**: Different scientific theories and perspectives
- **Contextual information**: Broader concepts that depend on understanding relationships

In a real-world application, you would replace these sample texts with your own documents - perhaps research papers, product manuals, company knowledge bases, or any other collection of texts you want to make searchable and queryable.

### Demonstrate use of this model

In [ ]:
texts = [
    # Factual content
    "The Earth is the third planet from the Sun and the only astronomical object known to harbor life.",
    "The average distance between the Earth and the Sun is about 149.6 million kilometers (93 million miles).",
    "Earth has one natural satellite, the Moon, which influences tides and stabilizes its axial tilt.",
    
    # Analytical context
    "The Earth's distance from the Sun, combined with its atmosphere, allows liquid water to exist, creating conditions suitable for life.",
    "If Earth were significantly closer to or farther from the Sun, its climate would be too hot or too cold for most known life forms.",
    
    # Opinion / theories
    "Theories about the origin of life on Earth include primordial soup theory, hydrothermal vent hypothesis, and panspermia, where life may have originated from space.",
    
    # Contextual / broader perspective
    "The Earth's position in the Solar System, within the habitable zone or 'Goldilocks Zone', is a key factor in making it suitable for life.",
    "The interplay of distance from the Sun, atmospheric composition, and magnetic field contributes to Earth's long-term habitability."
]
rag_system = AdaptiveRAG(texts)

## Testing Our Adaptive RAG System

Now comes the exciting part - testing our system with different types of questions to see how it adapts its approach. We'll ask four different questions that should trigger each of our four retrieval strategies:

1. **Factual Question**: "What is the distance between the Earth and the Sun?" - Should trigger the Factual strategy for a precise answer

2. **Analytical Question**: "How does the Earth's distance from the Sun affect its climate?" - Should trigger the Analytical strategy for comprehensive analysis

3. **Opinion Question**: "What are the different theories about the origin of life on Earth?" - Should trigger the Opinion strategy to present multiple perspectives

4. **Contextual Question**: "How does the Earth's position in the Solar System influence its habitability?" - Should trigger the Contextual strategy for situation-dependent understanding

Watch the output to see how the system classifies each question and which strategy it chooses. This demonstrates the adaptive nature of our RAG system in action.

### Showcase the four different types of queries

In [ ]:
factual_result = rag_system.answer("What is the distance between the Earth and the Sun?").content #type: ignore
print(f"Answer: {factual_result}")

analytical_result = rag_system.answer("How does the Earth's distance from the Sun affect its climate?").content #type: ignore
print(f"Answer: {analytical_result}")

opinion_result = rag_system.answer("What are the different theories about the origin of life on Earth?").content #type: ignore
print(f"Answer: {opinion_result}")

contextual_result = rag_system.answer("How does the Earth's position in the Solar System influence its habitability?").content #type: ignore
print(f"Answer: {contextual_result}")

## What You've Just Learned

Congratulations! You've just built and tested a sophisticated Adaptive RAG system. Here's what makes this system special:

**Intelligence in Retrieval**: Unlike traditional RAG systems that use the same approach for every question, your system intelligently adapts its search strategy based on what type of question is being asked.

**Real-World Applicability**: The four strategies (Factual, Analytical, Opinion, Contextual) cover the vast majority of question types you'll encounter in real applications.

**Scalability**: You can easily replace the sample texts with your own documents - whether they're product manuals, research papers, company knowledge bases, or any other text collection.

**Extensibility**: The modular design means you can easily add new retrieval strategies, modify existing ones, or integrate additional features like user personalization or domain-specific enhancements.


![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--adaptive-retrieval)